Instructions: So far we have worked on EDA. This lab will focus on data cleaning and wrangling from everything we noticed before.

We will start with removing outliers. So far, we have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.
Create a copy of the dataframe for the data wrangling.
Normalize the continuous variables. You can use any one method you want.
Encode the categorical variables
The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.
Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.
Hint for Categorical Variables

You should deal with the categorical variables as shown below (for ordinal encoding, dummy code has been provided as well):

In [1]:
#Import the necessary libraries.

import numpy as np
import scipy.stats as stats
from scipy.stats import norm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn import regplot
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Load the data
customer_df = pd.read_csv("../files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv")

In [3]:
#First look at its main features (head, shape, info).
customer_df.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [4]:
customer_df.shape

(9134, 24)

In [5]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Customer                       9134 non-null   object 
 1   State                          9134 non-null   object 
 2   Customer Lifetime Value        9134 non-null   float64
 3   Response                       9134 non-null   object 
 4   Coverage                       9134 non-null   object 
 5   Education                      9134 non-null   object 
 6   Effective To Date              9134 non-null   object 
 7   EmploymentStatus               9134 non-null   object 
 8   Gender                         9134 non-null   object 
 9   Income                         9134 non-null   int64  
 10  Location Code                  9134 non-null   object 
 11  Marital Status                 9134 non-null   object 
 12  Monthly Premium Auto           9134 non-null   i

In [6]:
#Rename the columns so they follow the PE8 (snake case).
standard_columns = [customer_df.columns[i].lower() for i in range(len(customer_df.columns))]
standard_columns = [col_name.lower().replace(' ', '_') for col_name in customer_df.columns]
customer_df.columns = standard_columns

In [10]:
mask = customer_df.isna()
mask

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9130,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9131,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9132,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
# Change effective to date column to datetime format.
customer_df['effective_to_date'] = pd.to_datetime(customer_df['effective_to_date'])
customer_df['effective_to_date']

0      2011-02-24
1      2011-01-31
2      2011-02-19
3      2011-01-20
4      2011-02-03
          ...    
9129   2011-02-10
9130   2011-02-12
9131   2011-02-06
9132   2011-02-03
9133   2011-02-14
Name: effective_to_date, Length: 9134, dtype: datetime64[ns]

In [12]:
# Check NaN values per column.
customer_df.isnull().sum()

customer                         0
state                            0
customer_lifetime_value          0
response                         0
coverage                         0
education                        0
effective_to_date                0
employmentstatus                 0
gender                           0
income                           0
location_code                    0
marital_status                   0
monthly_premium_auto             0
months_since_last_claim          0
months_since_policy_inception    0
number_of_open_complaints        0
number_of_policies               0
policy_type                      0
policy                           0
renew_offer_type                 0
sales_channel                    0
total_claim_amount               0
vehicle_class                    0
vehicle_size                     0
dtype: int64

In [13]:
# Define a function that differentiates between continuous and discrete variables. Hint: A number of unique values might be useful. 
# Store continuous data into a continuous variable and do the same for discrete and categorical.
#data = customer_df.copy()
def separate_variables(df):

    continuous_columns = []
    discrete_columns = []
    thresh_hold = 100

    for i in df.columns:
        if df[i].nunique() >= thresh_hold:
            continuous_columns.append(i)
        else:
            discrete_columns.append(i)
    continuous_variables = df[continuous_columns]
    discrete_variables = df[discrete_columns]
    return discrete_variables, continuous_variables


In [15]:
discrete, continuous   = separate_variables(customer_df)

In [16]:
discrete.dtypes

state                                object
response                             object
coverage                             object
education                            object
effective_to_date            datetime64[ns]
employmentstatus                     object
gender                               object
location_code                        object
marital_status                       object
months_since_last_claim               int64
number_of_open_complaints             int64
number_of_policies                    int64
policy_type                          object
policy                               object
renew_offer_type                     object
sales_channel                        object
vehicle_class                        object
vehicle_size                         object
dtype: object

In [111]:
for column in discrete:
    print(discrete[column].agg(['nunique', 'count','dtypes']),'\n')

nunique         5
count        9134
dtypes     object
Name: state, dtype: object 

nunique         2
count        9134
dtypes     object
Name: response, dtype: object 

nunique         3
count        9134
dtypes     object
Name: coverage, dtype: object 

nunique         5
count        9134
dtypes     object
Name: education, dtype: object 

nunique                59
count                9134
dtypes     datetime64[ns]
Name: effective_to_date, dtype: object 

nunique         5
count        9134
dtypes     object
Name: employmentstatus, dtype: object 

nunique         2
count        9134
dtypes     object
Name: gender, dtype: object 

nunique         3
count        9134
dtypes     object
Name: location_code, dtype: object 

nunique         3
count        9134
dtypes     object
Name: marital_status, dtype: object 

nunique       36
count       9134
dtypes     int64
Name: months_since_last_claim, dtype: object 

nunique        6
count       9134
dtypes     int64
Name: number_of_open_complain

In [113]:
#Get categorical features.
cat_features = discrete.select_dtypes('object')
cat_features

,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,Washington,No,Basic,Bachelor,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,Arizona,No,Extended,Bachelor,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,Nevada,No,Premium,Bachelor,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,California,No,Basic,Bachelor,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,Washington,No,Basic,Bachelor,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,California,No,Basic,Bachelor,Employed,M,Urban,Married,Personal Auto,Personal L1,Offer2,Web,Four-Door Car,Medsize
9130,California,Yes,Extended,College,Employed,F,Suburban,Divorced,Corporate Auto,Corporate L3,Offer1,Branch,Four-Door Car,Medsize
9131,California,No,Extended,Bachelor,Unemployed,M,Suburban,Single,Corporate Auto,Corporate L2,Offer1,Branch,Four-Door Car,Medsize
9132,California,No,Extended,College,Employed,M,Suburban,Married,Personal Auto,Personal L2,Offer3,Branch,Four-Door Car,Large


In [116]:
#What should we do with the customer_id column?
## We can make it the index of the continuous dataframe
continuous = continuous.set_index('customer')
continuous

,customer_lifetime_value,income,monthly_premium_auto,months_since_policy_inception,total_claim_amount
customer,,,,,
BU79786,2763.519279,56274,69,5,384.811147
QZ44356,6979.535903,0,94,42,1131.464935
AI49188,12887.431650,48767,108,38,566.472247
WW63253,7645.861827,0,106,65,529.881344
HB64268,2813.692575,43836,73,44,138.130879
...,...,...,...,...,...
LA72316,23405.987980,71941,73,89,198.234764
PK87824,3096.511217,21604,79,28,379.200000
TD14365,8163.890428,0,85,37,790.784983


In [83]:
#We will start with removing outliers. So far, we have discussed different methods to remove outliers. 
#Use the one you feel more comfortable with, define a function for that. 
#Use the function to remove the outliers and apply it to the dataframe.

#Function to removing outliers 
def remove_outliers(customer_df, threshold=1.5,in_columns=customer_df.select_dtypes(np.number).columns, skip_columns=[]):
    
    for column in in_columns:
        if column not in skip_columns:
            upper = np.percentile(customer_df[column],75)
            lower = np.percentile(customer_df[column],25)
            iqr = upper - lower
            upper_limit = upper + threshold * iqr
            lower_limit = lower - threshold * iqr
            df = customer_df[(customer_df[column]>lower_limit) & (customer_df[column]<upper_limit)]
    return customer_df

In [84]:
customer_no_outlier = remove_outliers(customer_df)
customer_no_outlier.head()

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2011-02-24,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,2011-01-31,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2011-02-19,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,2011-01-20,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2011-02-03,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [85]:
#Create a copy of the dataframe for the data wrangling.
customer_clean = customer_no_outlier.copy()
customer_clean = customer_clean.set_index('customer')
customer_clean.head()

,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,location_code,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
customer,,,,,,,,,,,,,,,,,,,,,
BU79786,Washington,2763.519279,No,Basic,Bachelor,2011-02-24,Employed,F,56274,Suburban,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
QZ44356,Arizona,6979.535903,No,Extended,Bachelor,2011-01-31,Unemployed,F,0,Suburban,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
AI49188,Nevada,12887.431650,No,Premium,Bachelor,2011-02-19,Employed,F,48767,Suburban,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
WW63253,California,7645.861827,No,Basic,Bachelor,2011-01-20,Unemployed,M,0,Suburban,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
HB64268,Washington,2813.692575,No,Basic,Bachelor,2011-02-03,Employed,M,43836,Rural,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [86]:
#Normalize the continuous variables. You can use any one method you want.

def separate_variables(df):

    continuous_columns = []
    discrete_columns = []
    thresh_hold = 100

    for i in df.columns:
        if df[i].nunique() >= thresh_hold:
            continuous_columns.append(i)
        else:
            discrete_columns.append(i)
    continuous_variables = df[continuous_columns]
    discrete_variables = df[discrete_columns]
    return discrete_variables, continuous_variables

In [87]:
discrete_customer, continuous_customer = separate_variables(customer_clean)
continuous_customer

,customer_lifetime_value,income,monthly_premium_auto,months_since_policy_inception,total_claim_amount
customer,,,,,
BU79786,2763.519279,56274,69,5,384.811147
QZ44356,6979.535903,0,94,42,1131.464935
AI49188,12887.431650,48767,108,38,566.472247
WW63253,7645.861827,0,106,65,529.881344
HB64268,2813.692575,43836,73,44,138.130879
...,...,...,...,...,...
LA72316,23405.987980,71941,73,89,198.234764
PK87824,3096.511217,21604,79,28,379.200000
TD14365,8163.890428,0,85,37,790.784983


In [88]:
#Using BoxCox transformation to normalize continuous variables
def boxcox_transform(df):
    numeric_cols = df.select_dtypes(np.number).columns
    _ci = {column: None for column in numeric_cols}
    for column in numeric_cols:
        # since i know any columns should take negative numbers, to avoid -inf in df
        df[column] = np.where(df[column]<=0, np.NAN, df[column]) 
        df[column] = df[column].fillna(df[column].mean())
        transformed_data, ci = stats.boxcox(df[column])
        df[column] = transformed_data
        _ci[column] = [ci] 
    return df, _ci

In [89]:
continuous_customer_norm, _ci = boxcox_transform(continuous_customer)
continuous_customer_norm

,customer_lifetime_value,income,monthly_premium_auto,months_since_policy_inception,total_claim_amount
customer,,,,,
BU79786,2.703839,1247.937066,0.685347,3.017887,33.035752
QZ44356,2.754926,1167.961720,0.685871,18.780377,56.357614
AI49188,2.780772,1144.063081,0.686039,17.389171,40.071789
WW63253,2.759125,1167.961720,0.686018,26.160800,38.763252
HB64268,2.704995,1072.375744,0.685461,19.461641,19.560154
...,...,...,...,...,...
LA72316,2.801170,1448.531450,0.685461,33.090025,23.588905
PK87824,2.711030,697.790558,0.685606,13.709541,32.792951
TD14365,2.762062,1167.961720,0.685725,17.034934,47.263726


In [90]:
#Encode the categorical variables
customer_cat = discrete_customer.select_dtypes('object')
customer_cat

,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
customer,,,,,,,,,,,,,,
BU79786,Washington,No,Basic,Bachelor,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
QZ44356,Arizona,No,Extended,Bachelor,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
AI49188,Nevada,No,Premium,Bachelor,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
WW63253,California,No,Basic,Bachelor,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
HB64268,Washington,No,Basic,Bachelor,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LA72316,California,No,Basic,Bachelor,Employed,M,Urban,Married,Personal Auto,Personal L1,Offer2,Web,Four-Door Car,Medsize
PK87824,California,Yes,Extended,College,Employed,F,Suburban,Divorced,Corporate Auto,Corporate L3,Offer1,Branch,Four-Door Car,Medsize
TD14365,California,No,Extended,Bachelor,Unemployed,M,Suburban,Single,Corporate Auto,Corporate L2,Offer1,Branch,Four-Door Car,Medsize


In [91]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='error', drop='first')
encoder.fit(customer_cat)

encoded = encoder.transform(customer_cat).toarray()
encoded #.shape # 

# X = np.concatenate([X_num, encoded], axis=1)
# X.shape

array([[0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 1., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 1., 1., 0.]])

In [92]:
customer_clean.dtypes

state                                    object
customer_lifetime_value                 float64
response                                 object
coverage                                 object
education                                object
effective_to_date                datetime64[ns]
employmentstatus                         object
gender                                   object
income                                    int64
location_code                            object
marital_status                           object
monthly_premium_auto                      int64
months_since_last_claim                   int64
months_since_policy_inception             int64
number_of_open_complaints                 int64
number_of_policies                        int64
policy_type                              object
policy                                   object
renew_offer_type                         object
sales_channel                            object
total_claim_amount                      

In [93]:
# The time variable can be useful. Try to transform its data into a useful one. 
# Hint: Day week and month as integers might be useful.
#discrete_customer[['effective_to_date']]

customer_clean['Month']=customer_clean['effective_to_date'].dt.month
customer_clean['Week']=customer_clean['effective_to_date'].dt.week
customer_clean['Day']=customer_clean['effective_to_date'].dt.day
customer_clean.head()

,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,location_code,...,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,Month,Week,Day
customer,,,,,,,,,,,,,,,,,,,,,
BU79786,Washington,2763.519279,No,Basic,Bachelor,2011-02-24,Employed,F,56274,Suburban,...,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize,2,8,24
QZ44356,Arizona,6979.535903,No,Extended,Bachelor,2011-01-31,Unemployed,F,0,Suburban,...,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize,1,5,31
AI49188,Nevada,12887.431650,No,Premium,Bachelor,2011-02-19,Employed,F,48767,Suburban,...,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize,2,7,19
WW63253,California,7645.861827,No,Basic,Bachelor,2011-01-20,Unemployed,M,0,Suburban,...,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize,1,3,20
HB64268,Washington,2813.692575,No,Basic,Bachelor,2011-02-03,Employed,M,43836,Rural,...,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize,2,5,3


In [94]:
customer_clean.dtypes

state                                    object
customer_lifetime_value                 float64
response                                 object
coverage                                 object
education                                object
effective_to_date                datetime64[ns]
employmentstatus                         object
gender                                   object
income                                    int64
location_code                            object
marital_status                           object
monthly_premium_auto                      int64
months_since_last_claim                   int64
months_since_policy_inception             int64
number_of_open_complaints                 int64
number_of_policies                        int64
policy_type                              object
policy                                   object
renew_offer_type                         object
sales_channel                            object
total_claim_amount                      

In [95]:
# Since the model will only accept numerical data, check and make sure that every column is numerical, 
#if some are not, change it using encoding. Hint for Categorical Variables
customer_nums = customer_clean.select_dtypes(np.number)
customer_cats = customer_clean.select_dtypes(np.object)
customer_nums


,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount,Month,Week,Day
customer,,,,,,,,,,,
BU79786,2763.519279,56274,69,32,5,0,1,384.811147,2,8,24
QZ44356,6979.535903,0,94,13,42,0,8,1131.464935,1,5,31
AI49188,12887.431650,48767,108,18,38,0,2,566.472247,2,7,19
WW63253,7645.861827,0,106,18,65,0,7,529.881344,1,3,20
HB64268,2813.692575,43836,73,12,44,0,1,138.130879,2,5,3
...,...,...,...,...,...,...,...,...,...,...,...
LA72316,23405.987980,71941,73,18,89,0,2,198.234764,2,6,10
PK87824,3096.511217,21604,79,14,28,0,1,379.200000,2,6,12
TD14365,8163.890428,0,85,9,37,3,2,790.784983,2,5,6


In [75]:
#You should deal with the categorical variables as shown below (for ordinal encoding, dummy code has been provided as well):

# One hot to state
# Ordinal to coverage
# Ordinal to employmentstatus
# Ordinal to location code
# One hot to marital status
# One hot to policy type
# One hot to policy
# One hot to renew offercustomer_df
# One hot to sales channel
# One hot vehicle class
# Ordinal vehicle size
#customer_cats

for col in customer_cats.columns:
    print(customer_cats[col].unique())

['Washington' 'Arizona' 'Nevada' 'California' 'Oregon']
['No' 'Yes']
['Basic' 'Extended' 'Premium']
['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']
['Employed' 'Unemployed' 'Medical Leave' 'Disabled' 'Retired']
['F' 'M']
['Suburban' 'Rural' 'Urban']
['Married' 'Single' 'Divorced']
['Corporate Auto' 'Personal Auto' 'Special Auto']
['Corporate L3' 'Personal L3' 'Corporate L2' 'Personal L1' 'Special L2'
 'Corporate L1' 'Personal L2' 'Special L1' 'Special L3']
['Offer1' 'Offer3' 'Offer2' 'Offer4']
['Agent' 'Call Center' 'Web' 'Branch']
['Two-Door Car' 'Four-Door Car' 'SUV' 'Luxury SUV' 'Sports Car'
 'Luxury Car']
['Medsize' 'Small' 'Large']


In [97]:
customer_cats["coverage"] = customer_cats["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
customer_cats["education"] = customer_cats["education"].map({"High School or Below" : 0, "College" : 1, "Bachelor" : 2,
                                                                          "Master" : 3, "Doctor": 4})
customer_cats["employmentstatus"] = customer_cats["employmentstatus"].map({"Unemployed" : 0, "Disabled" : 1, "Retired" : 2,
                                                                          "Medical Leave" : 3, "Employed": 4})
customer_cats["location_code"] = customer_cats["location_code"].map({"Rural" : 0, "Suburban" : 1, "Urban" : 2})
customer_cats["vehicle_size"] = customer_cats["vehicle_size"].map({"Small" : 0, "Medsize" : 1, "Large" : 2})
customer_cats.head()

,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
customer,,,,,,,,,,,,,,
BU79786,Washington,No,0,2,4,F,1,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,1
QZ44356,Arizona,No,1,2,0,F,1,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,1
AI49188,Nevada,No,2,2,4,F,1,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,1
WW63253,California,No,0,2,0,M,1,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,1
HB64268,Washington,No,0,2,4,M,0,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,1


In [99]:
customer_cats2 = customer_cats.select_dtypes(np.object)
customer_cats1 = customer_cats.select_dtypes(np.number)
customer_cats2

,state,response,gender,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class
customer,,,,,,,,,
BU79786,Washington,No,F,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car
QZ44356,Arizona,No,F,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car
AI49188,Nevada,No,F,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car
WW63253,California,No,M,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV
HB64268,Washington,No,M,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car
...,...,...,...,...,...,...,...,...,...
LA72316,California,No,M,Married,Personal Auto,Personal L1,Offer2,Web,Four-Door Car
PK87824,California,Yes,F,Divorced,Corporate Auto,Corporate L3,Offer1,Branch,Four-Door Car
TD14365,California,No,M,Single,Corporate Auto,Corporate L2,Offer1,Branch,Four-Door Car


In [103]:
encoder2 = OneHotEncoder(handle_unknown='error', drop='first')
encoder2.fit(customer_cats2)

encoded2 = encoder2.transform(customer_cats2).toarray()
encoded2.shape # 

(9134, 29)

In [107]:
customer_cats_encoded = np.concatenate([customer_cats1, encoded2], axis=1)
customer_cats_encoded.shape

(9134, 34)